# List of all symbol / tickers in US's major indices and exchanges

In this notebook you can get the list of all constituents / components and tickers in S&P500, S&P400, Dow Jones and NASDAQ100
Retrieved from wikipedia and Nasdaq websites

You'll get a dictionary with dataframes and csv files

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Get components / constituents of indices

In [2]:
#this code is stable as long as the following addresses are not being changed and thier structure not change
urls = {'SNP500':'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies',
'SNP400':'https://en.wikipedia.org/wiki/List_of_S%26P_400_companies',
'DJ':'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average',
'NASDAQ100':'http://www.nasdaq.com/quotes/nasdaq-100-stocks.aspx?render=download'}
indices = {}
for url in urls:
    if url == 'NASDAQ100':
        df = pd.read_csv(urls[url],usecols=[0,1],header=0)
        df.columns = ['Symbol','Name']
    else:
        result = requests.get(urls[url])
        c = result.content
        bs = BeautifulSoup(c,'lxml')
        table = str(bs.find('table',{'class':'wikitable sortable'}))
        df = pd.read_html(table)[0]
        df.columns = df.iloc[0] #set the first row as columns header
        df.drop(0,inplace=True) #drop the first row
        if url == 'DJ' :
            df = df[['Symbol','Company']]
            df.columns = ['Symbol','Name']
        elif url =='SNP500':
            df = df[['Ticker symbol','Security','CIK']]
            df.columns = ['Symbol','Name','cik']
        else:
            df = df[['Ticker Symbol','Company']]
            df.columns = ['Symbol','Name']
    df = df.rename(columns={'Symbol':'symbol','Name':'name'})
    df.to_csv(url + '.csv',index=False, encoding='utf-8')
    indices[url] = df.reset_index(drop=True)

## Get symbols of major exchanges in US

You'll get a dictionaty of dataframes, merged dataframe and csv files

In [3]:
urls = {'NYSE':'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download',
'AMEX':'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=AMEX&render=download',
'NASDAQ':'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download'}
exchanges = {i:pd.read_csv(urls[i],usecols=[0,1],header=0) for i in urls}

In [4]:
merged_exchanges = pd.DataFrame(columns=['Symbol'])
for i in exchanges:
    exchanges[i].columns = ['Symbol',i+'Name']
    exchanges[i].to_csv(i + '.csv',index=False, encoding='utf-8')
    merged_exchanges = merged_exchanges.merge(exchanges[i],'outer',['Symbol'], sort=True).fillna('')
merged_exchanges.to_csv('merged_exchanges.csv',index=False, encoding='utf-8')